In [1]:
# mac m1
import pandas as pd
import openpyxl
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
plt.rcParams["axes.unicode_minus"] = False 
rc("font", family="Arial Unicode MS") # Windows: Malgun Gothic 
# %matplotlib inline 
get_ipython().run_line_magic("matplotlib", "inline")
from tqdm import tqdm

### 지역 구분 기준 정하기
2018년도 (코로나 발생 전) 출퇴근 시기 이용객수 상위 25%인 지하철 역을 기준으로 보고자 함

In [2]:
# 2018년도 출퇴근 시기 이용객수가 상위 25%인 지하철역
df_출근 = pd.read_csv('df_2018_출근.csv')
df_퇴근 = pd.read_csv('df_2018_퇴근.csv')
df = pd.concat([df_출근,df_퇴근])
many_passengers_subway = list(df['지하철역'].drop_duplicates()) # 중복 제거
len(many_passengers_subway)

132

총 132개의 정류장을 바탕으로 그 부근에 위치한 버스 정류장을 살펴볼 것임.

In [3]:
df_bus_Wsub = pd.read_csv("버스_출근퇴근_총_인원정보와_근처_지하철(최종).csv", index_col=0)

위에서 꼽은 132개의 지하철 역 리스트 근처에 위치하는 버스 정류장 구분하기

In [4]:
df_bus_Wsub['상위 25% 포함여부'] = 0

In [5]:
for i in tqdm(range(len(df_bus_Wsub))):
    df_bus_Wsub['상위 25% 포함여부'][i] = df_bus_Wsub['name'][i] in many_passengers_subway
    # 한줄씩 추가

  0%|          | 0/1488704 [00:00<?, ?it/s]/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_18578/3828755079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bus_Wsub['상위 25% 포함여부'][i] = df_bus_Wsub['name'][i] in many_passengers_subway
100%|██████████| 1488704/1488704 [00:15<00:00, 94625.50it/s]


df_high : 상위 25% 승객수의 정류장에 해당하는 버스정류장만 남아있는 데이터 프레임

In [6]:
df_high = df_bus_Wsub[df_bus_Wsub['상위 25% 포함여부'] == True]
df_high.describe()

,사용년월,표준버스정류장ID,경도,위도,출근,퇴근,총 이동량,lat,lng,distance
count,576058.000000,5.760580e+05,576058.000000,576058.000000,576058.000000,576058.000000,576058.000000,576058.000000,576058.000000,576058.000000
mean,201955.841011,1.182882e+08,126.987883,37.530752,1206.474921,1307.031026,2513.505947,37.531247,126.987457,0.520203
std,113.521886,2.364694e+07,0.069677,0.056386,1836.345228,1892.325844,3129.900217,0.055364,0.069053,0.503862
min,201801.000000,1.000000e+08,126.747558,37.347408,0.000000,0.000000,0.000000,37.350077,126.753664,0.005742
25%,201812.000000,1.050004e+08,126.931462,37.488402,229.000000,234.000000,720.000000,37.486947,126.929715,0.211821
50%,201912.000000,1.160001e+08,126.987750,37.526956,714.000000,742.000000,1719.000000,37.527098,126.986325,0.396419
75%,202101.000000,1.210001e+08,127.039655,37.569485,1520.000000,1650.000000,3272.000000,37.566295,127.036456,0.649412
max,202112.000000,2.350003e+08,127.144647,37.751305,75532.000000,68938.000000,144470.000000,37.738899,127.132481,6.525692


distance : 버스 정류장과 역과의 거리
  
distance가 가장 먼 경우는 약 6.5km로 부근으로 설정하기 어려움.

In [7]:
df_high['distance'].quantile(q=0.9)

1.0115925625722275

distance의 데이터 90%는 지하철 역 반경 약 1km 이내에 위치해있는 것으로 보임. 따라서 거리 제한을 둔 데이터만 보기

In [8]:
df_high_distance = df_high[df_high['distance'] <= df_high['distance'].quantile(q=0.9)]
df_high_distance

,사용년월,노선번호,노선명,표준버스정류장ID,역명,경도,위도,출근,퇴근,총 이동량,line,name,lat,lng,distance,상위 25% 포함여부
0,201801,100,100번(하계동~용산구청),100000002,창경궁.서울대학교병원,126.996566,37.579183,2224,384,2608,04호선,혜화,37.582336,127.001844,0.582445,True
1,201801,102,102번(상계동~동대문),100000002,창경궁.서울대학교병원,126.996566,37.579183,2551,110,2661,04호선,혜화,37.582336,127.001844,0.582445,True
2,201801,104,104번(강북청소년수련관난나~숭례문),100000002,창경궁.서울대학교병원,126.996566,37.579183,2748,306,3054,04호선,혜화,37.582336,127.001844,0.582445,True
3,201801,106,106번(의정부~동대문),100000002,창경궁.서울대학교병원,126.996566,37.579183,2133,398,2531,04호선,혜화,37.582336,127.001844,0.582445,True
4,201801,107,107번(송현동~동대문),100000002,창경궁.서울대학교병원,126.996566,37.579183,2734,222,2956,04호선,혜화,37.582336,127.001844,0.582445,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488654,202112,9408,9408번(성남 분당~영등포),206000399,청솔마을,127.116403,37.352545,13,0,13,분당선,미금,37.350077,127.108910,0.716927,True
1488693,202112,1017,1017번(월계동~상왕십리),105000559,e편한세상청계센트럴포레아파트,127.030252,37.570682,324,122,446,01호선,신설동,37.575297,127.025087,0.685960,True
1488694,202112,147,147번(월계동~도곡동),105000559,e편한세상청계센트럴포레아파트,127.030252,37.570682,636,471,1107,01호선,신설동,37.575297,127.025087,0.685960,True
1488695,202112,202,202번(불암동~후암동),105000559,e편한세상청계센트럴포레아파트,127.030252,37.570682,753,692,1445,01호선,신설동,37.575297,127.025087,0.685960,True


In [9]:
# 사용년월별 각 버스정류장의 출근,퇴근,총 이동량
df_bus = df_high_distance.pivot_table(index=['사용년월','표준버스정류장ID','역명','line','name','lat','lng'],values=['출근', '퇴근', '총 이동량'],aggfunc=sum).reset_index()
df_bus

,사용년월,표준버스정류장ID,역명,line,name,lat,lng,총 이동량,출근,퇴근
0,201801,100000001,종로2가사거리,01호선,종로3가,37.571607,126.991806,3719,2686,1033
1,201801,100000002,창경궁.서울대학교병원,04호선,혜화,37.582336,127.001844,49548,33805,15743
2,201801,100000003,명륜3가.성대입구,04호선,혜화,37.582336,127.001844,61196,31984,29212
3,201801,100000004,종로2가.삼일교,01호선,종각,37.570161,126.982923,7776,3752,4024
4,201801,100000005,혜화동로터리,04호선,혜화,37.582336,127.001844,32691,23234,9457
...,...,...,...,...,...,...,...,...,...,...
100824,202112,225000155,산본1동입구,01호선,금정,37.372221,126.943429,2690,1011,1679
100825,202112,225000156,산본1동,01호선,금정,37.372221,126.943429,6474,4750,1724
100826,202112,225000157,산본고가도로,01호선,금정,37.372221,126.943429,3504,2128,1376
100827,202112,225000250,산본주유소,01호선,금정,37.372221,126.943429,1409,1208,201


In [10]:
df_bus.to_csv('사용년월별_버스정류장_근처지하철_출퇴근_총이동량.csv')